In [95]:
#必要なライブラリをインポート
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [2]:
#URL（東京都千代田区の賃貸住宅情報 検索結果の1ページ目） #東京都／千代田区 12万円以下 15分以内 15年以内 2階以上／室内洗濯機置場／バス・トイレ別 管理費・共益費込み 鉄筋系／鉄骨系／ブロック・その他
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=12.0&co=1&et=15&cn=15&mb=0&mt=9999999&kz=1&kz=2&kz=4&tc=0400101&tc=0400501&tc=0400301&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2='

#データ取得
result = requests.get(url)
c = result.content

#HTMLを元に、オブジェクトを作る
soup = BeautifulSoup(c, "html.parser")

In [3]:
#ページ数を取得
body = soup.find("body")
pages = body.find("div",{'class':'pagination pagination_set-nav'}) #Page数の部分のhtmlを抜き出す
links = pages.select("a[href]") #link付きaタグを抜き出す
#link_text = [link.get_text() for link in links] #Loopでテキスト部を抜き出す
#page_num = [int(v) for v in link_text if v.isdigit()] #数値に変換できるものだけ取り出す
#max_page = max(page_num) #最大値を取得

In [93]:
#ページ選択で数値になっているリンクを引っ張ってくる（"次へ"を除く）
PageURLs = [link.get("href") for link in links if link.get_text().isdigit()] 
#1ページ目を先頭に格納
PageURLs.insert(0, url)

In [8]:
#物件リストの部分を切り出し
summary = soup.find("div",{'id':'js-bukkenList'})
#マンション名、住所、立地（最寄駅/徒歩~分）、築年数、建物高さが入っているcassetteitemを全て抜き出し - デフォルト設定で最大30件の物件表示
cassetteitems = summary.find_all("div",{'class':'cassetteitem'})

In [75]:
#Loopを見据える マンションの数
i = 1

In [76]:
#マンション名取得
EstateName = cassetteitems[i].find("div",{'class':'cassetteitem_content-title'}).get_text()

In [77]:
#住所取得
EstateAddress = cassetteitems[i].find("li",{'class':'cassetteitem_detail-col1'}).get_text()

In [91]:
#最寄り駅取得
EstateLocationElem = cassetteitems[i].find("li",{'class':'cassetteitem_detail-col2'}).find_all("div",{'class':'cassetteitem_detail-text'})
EstateLocations = [EstateLocation.get_text() for EstateLocation in EstateLocationElem] #リストで取得
EstateLocation = ' --- '.join(EstateLocations)

In [79]:
#築年数と建物高さを取得
EstateCol3Elem = cassetteitems[i].find("li",{'class':'cassetteitem_detail-col3'}).find_all("div")
EstageAge = EstateCol3Elem[0].get_text()
EstageHight = EstateCol3Elem[1].get_text()

In [101]:
#Room Header Name
RoomtableHeadElem = RoomtableElem.find("thead").find_all("th")
HeaderNames = [temp.get_text() for temp in RoomtableHeadElem]
HeaderNames.append("NewArrival")
HeaderNames.extend(["マンション名", "住所", "最寄り駅", "築年数", "建物高さ"])
len(HeaderNames)

15

In [102]:
#Header Info をListに
HeaderInfo = [EstateName, EstateAddress, EstateLocation, EstageAge, EstageHight]
#階、賃料、管理費、敷/礼/保証/敷引,償却、間取り、専有面積が入っているtableを全て抜き出し
RoomtableElem =  cassetteitems[i].find("table",{'class':'cassetteitem_other'})
RoomDetail = []
#Contents 
for rooms in RoomtableElem.find_all("tbody"):
    Roomtable = [temp.get_text() for temp in rooms.findAll('td')]
    if "cassetteitem_other-checkbox--newarrival" in rooms.td['class']:
        Roomtable.append("New")
    else:
        Roomtable.append("Exsiting")
    #Add Header Info
    Roomtable.extend(HeaderInfo)
    RoomDetail.append(Roomtable)
len(RoomDetail[0])

15

http://zdassen.hatenablog.com/entry/2018/04/07/234016

In [ ]:
#各建物から売りに出ている部屋数を取得
tbodies = cassetteitems[0].find_all('tbody')
print("部屋数: " + str(len(tbodies)))

#部屋の部分を切り出し
summary = soup.find("div",{'id':'js-bukkenList'})

In [ ]:
name = [] #マンション名
address = [] #住所
locations0 = [] #立地1つ目（最寄駅/徒歩~分）
locations1 = [] #立地2つ目（最寄駅/徒歩~分）
locations2 = [] #立地3つ目（最寄駅/徒歩~分）
age = [] #築年数
height = [] #建物高さ
floor = [] #階
rent = [] #賃料
admin = [] #管理費
others = [] #敷/礼/保証/敷引,償却
floor_plan = [] #間取り
area = [] #専有面積
    
    #各cassetteitemsに対し、以下の動作をループ
    for i in range(len(cassetteitems)):
        #各建物から売りに出ている部屋数を取得
        tbodies = cassetteitems[i].find_all('tbody')
        
        #マンション名取得
        subtitle = cassetteitems[i].find_all("div",{
            'class':'cassetteitem_content-title'})
        subtitle = str(subtitle)
        subtitle_rep = subtitle.replace(
            '[<div class="cassetteitem_content-title">', '')
        subtitle_rep2 = subtitle_rep.replace(
            '</div>]', '')

        #住所取得
        subaddress = cassetteitems[i].find_all("li",{
            'class':'cassetteitem_detail-col1'})
        subaddress = str(subaddress)
        subaddress_rep = subaddress.replace(
            '[<li class="cassetteitem_detail-col1">', '')
        subaddress_rep2 = subaddress_rep.replace(
            '</li>]', '')
        
        #部屋数だけ、マンション名と住所を繰り返しリストに格納（部屋情報と数を合致させるため）
        for y in range(len(tbodies)):
            name.append(subtitle_rep2)
            address.append(subaddress_rep2)

        #立地を取得
        sublocations = cassetteitems[i].find_all("li",{
            'class':'cassetteitem_detail-col2'})
        
        #立地は、1つ目から3つ目までを取得（4つ目以降は無視）
        for x in sublocations:
            cols = x.find_all('div')
            for i in range(len(cols)):
                text = cols[i].find(text=True)
                for y in range(len(tbodies)):
                    if i == 0:
                        locations0.append(text)
                    elif i == 1:
                        locations1.append(text)
                    elif i == 2:
                        locations2.append(text)
                        
        #築年数と建物高さを取得
        tbodies = cassetteitems[i].find_all('tbody')
        col3 = cassetteitems[i].find_all("li",{
            'class':'cassetteitem_detail-col3'})
        for x in col3:
            cols = x.find_all('div')
            for i in range(len(cols)):
                text = cols[i].find(text=True)
                for y in range(len(tbodies)):
                    if i == 0:
                        age.append(text)
                    else:
                        height.append(text)

    #階、賃料、管理費、敷/礼/保証/敷引,償却、間取り、専有面積が入っているtableを全て抜き出し
    tables = summary.find_all('table')

    #各建物（table）に対して、売りに出ている部屋（row）を取得
    rows = []
    for i in range(len(tables)):
        rows.append(tables[i].find_all('tr'))

    #各部屋に対して、tableに入っているtext情報を取得し、dataリストに格納
    data = []
    for row in rows:
        for tr in row:
            cols = tr.find_all('td')
            for td in cols:
                text = td.find(text=True)
                data.append(text)

    #dataリストから、階、賃料、管理費、敷/礼/保証/敷引,償却、間取り、専有面積を順番に取り出す
    index = 0
    for item in data:
        if '階' in item:
            floor.append(data[index])
            rent.append(data[index+1])
            admin.append(data[index+2])
            others.append(data[index+3])
            floor_plan.append(data[index+4])
            area.append(data[index+5])
        index +=1

In [ ]:
result = pages.select("a[href]") 

url = "https://www.jitec.ipa.go.jp/1_04hanni_sukiru/mondai_kaitou_2018h30.html" 
res = req.urlopen(url) 
soup = BeautifulSoup(res, "html.parser") 
result = soup.select("a[href]") 
print(result)

link_list =[] 
for link in result: 
    href = link.get("href") 
    link_list.append(href) 
    print(link_list)

pages2 = pages.find_all("a")

In [ ]:
#物件リストの部分を切り出し
summary = soup.find("div",{'id':'js-bukkenList'})

In [ ]:
summary

In [ ]:
pages_split = pages_text.split('</a></li>\n</ol>')
pages_split0 = pages_split[0]
pages_split1 = pages_split0[-3:]
pages_split2 = pages_split1.replace('>','')
pages_split2

In [ ]:





#URLを入れるリスト
urls = []

#1ページ目を格納
urls.append(url)

#2ページ目から最後のページまでを格納
for i in range(pages_split3-1):
    pg = str(i+2)
    url_page = url + '&pn=' + pg
    urls.append(url_page)
    
    
name = [] #マンション名
address = [] #住所
locations0 = [] #立地1つ目（最寄駅/徒歩~分）
locations1 = [] #立地2つ目（最寄駅/徒歩~分）
locations2 = [] #立地3つ目（最寄駅/徒歩~分）
age = [] #築年数
height = [] #建物高さ
floor = [] #階
rent = [] #賃料
admin = [] #管理費
others = [] #敷/礼/保証/敷引,償却
floor_plan = [] #間取り
area = [] #専有面積

In [ ]:
#各ページで以下の動作をループ
for url in urls:
    #物件リストを切り出し
    result = requests.get(url)
    c = result.content
    soup = BeautifulSoup(c)
    summary = soup.find("div",{'id':'js-bukkenList'})
    
    #マンション名、住所、立地（最寄駅/徒歩~分）、築年数、建物高さが入っているcassetteitemを全て抜き出し
    cassetteitems = summary.find_all("div",{'class':'cassetteitem'})

    #各cassetteitemsに対し、以下の動作をループ
    for i in range(len(cassetteitems)):
        #各建物から売りに出ている部屋数を取得
        tbodies = cassetteitems[i].find_all('tbody')
        
        #マンション名取得
        subtitle = cassetteitems[i].find_all("div",{
            'class':'cassetteitem_content-title'})
        subtitle = str(subtitle)
        subtitle_rep = subtitle.replace(
            '[<div class="cassetteitem_content-title">', '')
        subtitle_rep2 = subtitle_rep.replace(
            '</div>]', '')

        #住所取得
        subaddress = cassetteitems[i].find_all("li",{
            'class':'cassetteitem_detail-col1'})
        subaddress = str(subaddress)
        subaddress_rep = subaddress.replace(
            '[<li class="cassetteitem_detail-col1">', '')
        subaddress_rep2 = subaddress_rep.replace(
            '</li>]', '')
        
        #部屋数だけ、マンション名と住所を繰り返しリストに格納（部屋情報と数を合致させるため）
        for y in range(len(tbodies)):
            name.append(subtitle_rep2)
            address.append(subaddress_rep2)

        #立地を取得
        sublocations = cassetteitems[i].find_all("li",{
            'class':'cassetteitem_detail-col2'})
        
        #立地は、1つ目から3つ目までを取得（4つ目以降は無視）
        for x in sublocations:
            cols = x.find_all('div')
            for i in range(len(cols)):
                text = cols[i].find(text=True)
                for y in range(len(tbodies)):
                    if i == 0:
                        locations0.append(text)
                    elif i == 1:
                        locations1.append(text)
                    elif i == 2:
                        locations2.append(text)
                        
        #築年数と建物高さを取得
        tbodies = cassetteitems[i].find_all('tbody')
        col3 = cassetteitems[i].find_all("li",{
            'class':'cassetteitem_detail-col3'})
        for x in col3:
            cols = x.find_all('div')
            for i in range(len(cols)):
                text = cols[i].find(text=True)
                for y in range(len(tbodies)):
                    if i == 0:
                        age.append(text)
                    else:
                        height.append(text)

    #階、賃料、管理費、敷/礼/保証/敷引,償却、間取り、専有面積が入っているtableを全て抜き出し
    tables = summary.find_all('table')

    #各建物（table）に対して、売りに出ている部屋（row）を取得
    rows = []
    for i in range(len(tables)):
        rows.append(tables[i].find_all('tr'))

    #各部屋に対して、tableに入っているtext情報を取得し、dataリストに格納
    data = []
    for row in rows:
        for tr in row:
            cols = tr.find_all('td')
            for td in cols:
                text = td.find(text=True)
                data.append(text)

    #dataリストから、階、賃料、管理費、敷/礼/保証/敷引,償却、間取り、専有面積を順番に取り出す
    index = 0
    for item in data:
        if '階' in item:
            floor.append(data[index])
            rent.append(data[index+1])
            admin.append(data[index+2])
            others.append(data[index+3])
            floor_plan.append(data[index+4])
            area.append(data[index+5])
        index +=1
    
    #プログラムを10秒間停止する（スクレイピングマナー）
    time.sleep(10)